In [1]:
import os
import pandas as pd
from frame import Frame
from testing import z_score_trading
from google.cloud import storage
from google.cloud import bigquery

data=Frame()
stock_price=data.dataset()
#
# #importing FTSE 100 data from Google cloud
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../data/" + 'lewagon-statistical-arbitrage-ae470f7dcd48.json'
client = storage.Client()
bucket = client.get_bucket('stat_arb')
# #
# Set your GCP project details
project_id = "lewagon-statistical-arbitrage"
dataset_id = "FTSE_100_main"  # Replace with your dataset name
table_name = "FTSE100_csv"

# Construct full table path
table_id = f"{project_id}.{dataset_id}.{table_name}"
query = "SELECT * FROM `lewagon-statistical-arbitrage.FTSE_100_main.FTSE100_csv` ORDER BY Unnamed_0 ASC"
client = bigquery.Client(project=project_id)
FTSE100 = client.query(query).to_dataframe()

FTSE100.rename(columns={'Unnamed_0': 'Date',"close": "FTSE price"}, inplace=True)
FTSE100.set_index('Date', inplace=True)
FTSE100_close_price = pd.DataFrame(FTSE100["FTSE price"])
FTSE100_close_price

# #PCA function

# Specifically for FTSE data!!! Needs to be adjusted for other dataframes
#
os.chdir('..')
cwd=os.getcwd()

daily_weight = pd.read_csv(cwd + "/data/daily_weights.csv")
# daily_weight = daily_weight.rename(columns = lambda x : str(x)[:-2])
daily_weight = daily_weight.rename(columns={'Date': 'date'})
daily_weight["date"] = pd.to_datetime(daily_weight["date"])

# #z-score trading simulation
# #
pca_weights_df=daily_weight
underlying_df=stock_price
target_df=FTSE100_close_price
cal_days=60
trade_days=30


Fetching and Merging...
Merging completed successfully!
Started prepocessing....


/home/shalagodor/code/shalagodor/arb/stock-stat-replica/app/frame.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest.dropna(inplace=True)


Completed prepocessing.


In [2]:
import numpy as np

In [3]:
stock_aligned = pca_weights_df[['date']].merge(underlying_df, on='date')
weight_aligned = stock_aligned[["date"]].merge(pca_weights_df,on="date")
weight_aligned.set_index("date",inplace=True)
stock_aligned.set_index("date",inplace=True)

In [4]:
for name in stock_aligned.columns:
    if name not in weight_aligned.columns:
        stock_aligned.drop(name,axis=1,inplace=True)


In [5]:
investment_aligned=weight_aligned.join(target_df)
target_df = investment_aligned['FTSE price']

In [6]:
replication_aligned = weight_aligned.mul(investment_aligned['FTSE price'], axis=0)
weight_position = replication_aligned/stock_aligned

In [7]:
replications_df=pd.DataFrame(columns=range(cal_days+trade_days), dtype=float)

In [19]:
investment_aligned['FTSE price']

date
2022-07-15   NaN
2022-07-18   NaN
2022-07-19   NaN
2022-07-20   NaN
2022-07-21   NaN
              ..
2025-03-03   NaN
2025-03-04   NaN
2025-03-05   NaN
2025-03-06   NaN
2025-03-07   NaN
Name: FTSE price, Length: 668, dtype: float64

In [12]:
for i,r in weight_position.reset_index().iterrows():
    if i>cal_days:
        print(r)
        try_1 = r * stock_aligned[i-cal_days-1:min(i+trade_days-1, len(weight_position))]
        replication_index=pd.DataFrame(try_1.sum(axis=1).reset_index(drop=True))
        replications_df=pd.concat([replications_df,replication_index.T], axis=0)

date   2022-10-12
AAF           NaT
AAL           NaT
ABF           NaT
ADM           NaT
          ...    
UU            NaT
VOD           NaT
WEIR          NaT
WPP           NaT
WTB           NaT
Name: 61, Length: 100, dtype: datetime64[ns]


TypeError: cannot perform __mul__ with this index type: DatetimeArray

In [ ]:
stock_aligned = pca_weights_df[['date']].merge(underlying_df, on='date')
weight_aligned = stock_aligned[["date"]].merge(pca_weights_df,on="date")
weight_aligned.set_index("date",inplace=True)
stock_aligned.set_index("date",inplace=True)

for name in stock_aligned.columns:
    if name not in weight_aligned.columns:
        stock_aligned.drop(name,axis=1,inplace=True)

#target_df.index = pd.to_datetime(target_df.index)
investment_aligned=weight_aligned.join(target_df)
target_df = investment_aligned['FTSE price']

#need to make this flexible here!! FTSE price
#
replication_aligned = weight_aligned.mul(investment_aligned['FTSE price'], axis=0)
weight_position = replication_aligned/stock_aligned

# #Calculating replication portfolio for cal_days+trade_days days based on Date PCA
# #
replications_df=pd.DataFrame(columns=range(cal_days+trade_days), dtype=float)

for i,r in weight_position.reset_index().iterrows():
    if i>cal_days:
        try_1 = r * stock_aligned[i-cal_days-1:min(i+trade_days-1, len(weight_position))]
        replication_index=pd.DataFrame(try_1.sum(axis=1).reset_index(drop=True))
        replications_df=pd.concat([replications_df,replication_index.T], axis=0)

replications_df.index=weight_position.index[cal_days+1:]
replications_df.columns=[f'Calibration Day {i-cal_days}' if i < (cal_days-1) else f'Trading Day {i-cal_days+1}' for i in range(0, cal_days+trade_days,1)]
replications_df=replications_df.astype(float)

# #Calculating target for cal_days+trade_days days from Date
target_match_df=pd.DataFrame(columns=range(cal_days+trade_days,1))

ValueError: cannot reindex on an axis with duplicate labels

In [12]:
target_match_df

""


In [13]:
for i,r in weight_position.reset_index().iterrows():
        if i > cal_days:
            target_match=pd.DataFrame(target_df.iloc[i-cal_days-1:min(i+trade_days-1, len(weight_position))].reset_index(drop=True))
            target_match_df=pd.concat([target_match_df,target_match.T], axis=0)


In [15]:
target_match_df.index=weight_position.index[cal_days+1:]
target_match_df.columns=[f'Calibration Day {i-cal_days}' if i < (cal_days-1) else f'Trading Day {i-cal_days+1}' for i in range(0, cal_days+trade_days,1)]
target_match_df=target_match_df.astype(float)

In [16]:
target_match_df

,Calibration Day -60,Calibration Day -59,Calibration Day -58,Calibration Day -57,Calibration Day -56,Calibration Day -55,Calibration Day -54,Calibration Day -53,Calibration Day -52,Calibration Day -51,...,Trading Day 21,Trading Day 22,Trading Day 23,Trading Day 24,Trading Day 25,Trading Day 26,Trading Day 27,Trading Day 28,Trading Day 29,Trading Day 30
date,,,,,,,,,,,,,,,,,,,,,
2022-10-12,7159.000000,7223.200195,7296.299805,7264.299805,7270.500000,7276.399902,7306.299805,7306.299805,7348.200195,7345.299805,...,7306.100098,7296.299805,7375.299805,7318.000000,7385.200195,7369.399902,7351.200195,7346.500000,7385.500000,7376.899902
2022-10-13,7223.200195,7296.299805,7264.299805,7270.500000,7276.399902,7306.299805,7306.299805,7348.200195,7345.299805,7423.399902,...,7296.299805,7375.299805,7318.000000,7385.200195,7369.399902,7351.200195,7346.500000,7385.500000,7376.899902,7452.799805
2022-10-14,7296.299805,7264.299805,7270.500000,7276.399902,7306.299805,7306.299805,7348.200195,7345.299805,7423.399902,7413.399902,...,7375.299805,7318.000000,7385.200195,7369.399902,7351.200195,7346.500000,7385.500000,7376.899902,7452.799805,7465.200195
2022-10-17,7264.299805,7270.500000,7276.399902,7306.299805,7306.299805,7348.200195,7345.299805,7423.399902,7413.399902,7409.100098,...,7318.000000,7385.200195,7369.399902,7351.200195,7346.500000,7385.500000,7376.899902,7452.799805,7465.200195,7466.600098
2022-10-18,7270.500000,7276.399902,7306.299805,7306.299805,7348.200195,7345.299805,7423.399902,7413.399902,7409.100098,7445.700195,...,7385.200195,7369.399902,7351.200195,7346.500000,7385.500000,7376.899902,7452.799805,7465.200195,7466.600098,7486.700195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03,8359.400391,8335.799805,8349.400391,8308.599609,8352.099609,8280.400391,8301.599609,8311.799805,8300.299805,8262.099609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-04,8335.799805,8349.400391,8308.599609,8352.099609,8280.400391,8301.599609,8311.799805,8300.299805,8262.099609,8195.200195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05,8349.400391,8308.599609,8352.099609,8280.400391,8301.599609,8311.799805,8300.299805,8262.099609,8195.200195,8199.099609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#log_return calculation of the target and replication
#
target_log_returns=np.log(target_match_df/target_match_df.shift(1, axis=1))
replications_log_returns=np.log(replications_df/replications_df.shift(1, axis=1))

In [19]:
spread_past_df=pd.DataFrame(target_log_returns.iloc[:,1:cal_days+1].values-replications_log_returns.iloc[:,1:cal_days+1].values, index=target_log_returns.index)
spread_mean=spread_past_df.mean(axis=1)
spread_vol=spread_past_df.std(axis=1)

In [20]:
z_scores_df=pd.DataFrame((spread_past_df.iloc[:,-1]-spread_mean)/spread_vol, columns=['z_score'])

In [21]:
pos_low_threshold=0.5
pos_high_threshold=2
neg_low_threshold=-2
neg_high_threshold=-0.5
#
for i, r in z_scores_df.iterrows():
    if r['z_score'] > pos_low_threshold:
        if r['z_score'] <pos_high_threshold:
            z_scores_df.loc[i, 'direction']=-1
        else:
            z_scores_df.loc[i,'direction']=0
    elif r['z_score'] > neg_low_threshold:
        if r['z_score']<neg_high_threshold:
            z_scores_df.loc[i, 'direction']=1
        else: z_scores_df.loc[i, 'direction']=0

print(z_scores_df)

             z_score  direction
date                           
2022-10-12 -0.399399        0.0
2022-10-13 -0.038065        0.0
2022-10-14 -0.702430        1.0
2022-10-17 -1.552750        1.0
2022-10-18 -1.854670        1.0
...              ...        ...
2025-03-03  0.161474        0.0
2025-03-04 -1.310994        1.0
2025-03-05  0.924985       -1.0
2025-03-06 -0.679171        1.0
2025-03-07  0.608759       -1.0

[607 rows x 2 columns]


In [25]:
spread_df=pd.DataFrame(target_log_returns.iloc[:,cal_days+1:].values-replications_log_returns.iloc[:,cal_days+1:].values, index=target_log_returns.index)
spread_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
date,,,,,,,,,,,,,,,,,,,,,
2022-10-12,0.003403,-0.001934,-0.010959,-0.008817,0.007400,-0.000824,0.007634,0.003999,-0.012725,-0.004526,...,-0.007817,-0.008465,-0.013715,0.013088,-0.006255,-0.001585,-0.006099,-0.008405,-0.009712,-0.007454
2022-10-13,-0.000266,-0.008481,-0.008441,0.008103,-0.000480,0.008412,0.003509,-0.011353,-0.003781,-0.001560,...,-0.009105,-0.014101,0.009638,-0.005576,-0.003287,-0.005414,-0.008114,-0.008382,-0.005560,0.000761
2022-10-14,-0.011766,-0.010163,0.006045,-0.001341,0.007798,0.005572,-0.013818,-0.004810,0.001681,-0.005403,...,-0.014421,0.009589,-0.007216,-0.000622,-0.005321,-0.007367,-0.010706,-0.006928,0.001135,0.000273
2022-10-17,-0.018769,0.006021,0.006305,0.002698,0.009290,-0.017468,-0.015359,0.001388,-0.003641,-0.011511,...,0.001498,-0.002489,0.003253,0.000623,-0.006400,-0.014445,-0.007326,0.004249,-0.004652,-0.008124
2022-10-18,0.006579,0.005825,0.003144,0.009622,-0.017524,-0.014871,0.001249,-0.003404,-0.012262,-0.003368,...,-0.002668,0.002530,0.000298,-0.007237,-0.014312,-0.007380,0.004077,-0.004355,-0.008085,0.009371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03,-0.006222,0.005754,-0.002557,0.007280,0.007094,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-04,0.009574,-0.003023,0.006495,0.009317,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05,-0.003728,0.005350,0.007162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
z_scores_df['target entry']=target_df
z_scores_df['replication entry']=replications_df['Trading Day 1']

In [35]:
z_scores_df

,z_score,direction,target entry,replication entry
date,,,,
2022-10-12,-0.399399,0.0,6826.200195,6908.670854
2022-10-13,-0.038065,0.0,6850.299805,6824.386592
2022-10-14,-0.702430,1.0,6858.799805,6770.171141
2022-10-17,-1.552750,1.0,6920.200195,6731.266140
2022-10-18,-1.854670,1.0,6936.700195,6965.874822
...,...,...,...,...
2025-03-03,0.161474,0.0,8871.299805,8755.059422
2025-03-04,-1.310994,1.0,8759.000000,8956.643635
2025-03-05,0.924985,-1.0,8755.799805,8726.404085


In [36]:
for day in np.arange(0,trade_days-1, 1):
    single_score_df=pd.DataFrame(((spread_df.iloc[:,day]-spread_mean)/spread_vol), columns=[f'Day {day+1}'])
    z_scores_df=pd.concat((z_scores_df,single_score_df), axis=1)


In [37]:
z_scores_df

,z_score,direction,target entry,replication entry,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,...,Day 20,Day 21,Day 22,Day 23,Day 24,Day 25,Day 26,Day 27,Day 28,Day 29
date,,,,,,,,,,,,,,,,,,,,,
2022-10-12,-0.399399,0.0,6826.200195,6908.670854,0.159261,-0.397622,-1.339364,-1.115823,0.576321,-0.281792,...,-1.011466,-1.079113,-1.626969,1.169906,-0.848575,-0.361190,-0.832236,-1.072839,-1.209289,-0.973622
2022-10-13,-0.038065,0.0,6850.299805,6824.386592,-0.199904,-1.075512,-1.071222,0.692044,-0.222723,0.724972,...,-1.142059,-1.674512,0.855717,-0.765953,-0.521947,-0.748677,-1.036438,-1.064952,-0.764154,-0.090422
2022-10-14,-0.702430,1.0,6858.799805,6770.171141,-1.522592,-1.351658,0.376198,-0.411231,0.563047,0.325809,...,-1.805663,0.753998,-1.037562,-0.334572,-0.835463,-1.053640,-1.409553,-1.006847,-0.147202,-0.239095
2022-10-17,-1.552750,1.0,6920.200195,6731.266140,-1.982932,0.261691,0.287371,-0.039187,0.557636,-1.865098,...,-0.147899,-0.508838,0.010991,-0.227090,-0.862952,-1.591412,-0.946863,0.101198,-0.704727,-1.019106
2022-10-18,-1.854670,1.0,6936.700195,6965.874822,0.334200,0.268049,0.032828,0.601280,-1.780896,-1.548063,...,-0.477174,-0.021032,-0.216973,-0.878155,-1.498974,-0.890673,0.114645,-0.625240,-0.952606,0.579237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03,0.161474,0.0,8871.299805,8755.059422,-1.448586,1.357993,-0.589675,1.715774,1.672135,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-04,-1.310994,1.0,8759.000000,8956.643635,2.152008,-0.732910,1.446831,2.092983,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05,0.924985,-1.0,8755.799805,8726.404085,-0.789417,1.146764,1.533026,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
exit_day=trade_days-1

for start_date, row in z_scores_df.iterrows():
    test_date=pd.to_datetime(start_date).strftime('%Y-%m-%d')
    z_scores_df.loc[test_date, 'replication exit']=replications_df.loc[test_date, f'Trading Day {exit_day}']
    z_scores_df.loc[test_date, 'target exit']=target_match_df.loc[test_date, f'Trading Day {exit_day}']


In [39]:
z_scores_df

,z_score,direction,target entry,replication entry,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,...,Day 22,Day 23,Day 24,Day 25,Day 26,Day 27,Day 28,Day 29,replication exit,target exit
date,,,,,,,,,,,,,,,,,,,,,
2022-10-12,-0.399399,0.0,6826.200195,6908.670854,0.159261,-0.397622,-1.339364,-1.115823,0.576321,-0.281792,...,-1.626969,1.169906,-0.848575,-0.361190,-0.832236,-1.072839,-1.209289,-0.973622,7980.913326,7385.500000
2022-10-13,-0.038065,0.0,6850.299805,6824.386592,-0.199904,-1.075512,-1.071222,0.692044,-0.222723,0.724972,...,0.855717,-0.765953,-0.521947,-0.748677,-1.036438,-1.064952,-0.764154,-0.090422,7972.791552,7376.899902
2022-10-14,-0.702430,1.0,6858.799805,6770.171141,-1.522592,-1.351658,0.376198,-0.411231,0.563047,0.325809,...,-1.037562,-0.334572,-0.835463,-1.053640,-1.409553,-1.006847,-0.147202,-0.239095,7948.386105,7452.799805
2022-10-17,-1.552750,1.0,6920.200195,6731.266140,-1.982932,0.261691,0.287371,-0.039187,0.557636,-1.865098,...,0.010991,-0.227090,-0.862952,-1.591412,-0.946863,0.101198,-0.704727,-1.019106,8101.809761,7465.200195
2022-10-18,-1.854670,1.0,6936.700195,6965.874822,0.334200,0.268049,0.032828,0.601280,-1.780896,-1.548063,...,-0.216973,-0.878155,-1.498974,-0.890673,0.114645,-0.625240,-0.952606,0.579237,8232.316556,7466.600098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03,0.161474,0.0,8871.299805,8755.059422,-1.448586,1.357993,-0.589675,1.715774,1.672135,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-04,-1.310994,1.0,8759.000000,8956.643635,2.152008,-0.732910,1.446831,2.092983,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05,0.924985,-1.0,8755.799805,8726.404085,-0.789417,1.146764,1.533026,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
weight_aligned.loc['2022-10-14'][weight_aligned.loc['2022-10-14']>0]

BATS    0.078583
BTA     0.071269
CNA     0.153768
GSK     0.075785
NG      0.120206
SHEL    0.084454
SSE     0.155918
SVT     0.084728
TSCO    0.071470
UU      0.103819
Name: 2022-10-14 00:00:00, dtype: float64